In [1]:
!pip -q install groq pypdf

In [2]:
import os
os.environ["GROQ_API_KEY"] = "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"
os.environ["TOMTOM_API_KEY"] = "lsMxbkPAAw9Zoya1rFoe9WEZGUBJnAee"


In [3]:
import requests, os

key = os.getenv("TOMTOM_API_KEY")
url = "https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
params = {
    "key": key,
    "point": "41.0369,28.9850"  # Taksim civarı
}

r = requests.get(url, params=params, timeout=20)
print("Status:", r.status_code)
print(r.text[:300])


Status: 200
{"flowSegmentData":{"frc":"FRC2","currentSpeed":17,"freeFlowSpeed":33,"currentTravelTime":2309,"freeFlowTravelTime":1189,"confidence":1,"roadClosure":false,"coordinates":{"coordinate":[{"latitude":41.003728529171610,"longitude":28.955802137254693},{"latitude":41.003780841406027,"longitude":28.955649


In [4]:

# AKILLI ULAŞIM & TRAFİK OPTİMİZASYONU
# ReAct + Multi-RAG + API Tools (TEK PARÇA KOD)
# ============================================================
# - Agent.pdf doğrultusunda ReAct mimarisi (Thought->Action->Observation)
# - Multi-RAG: SAFETY / REGULATION / SIGNS (PDF'lerden)
# - API Tool: canlı trafik yoğunluğu (TomTom örneği) + fallback mock
# - Math Tool: hesaplama
# - Benchmark: RAG yetmeyen promptlar -> API'ye bağlanmalı
#
# Not: RAG tool "cevap" üretmez; ham context döndürür. (Agent.pdf)
# ============================================================

import os
import re
import json
import math
import time
import random
import requests
import ast
from dataclasses import dataclass
from typing import List, Dict, Any, Tuple, Optional

# ----------------------------
# (1) Opsiyonel: Groq LLM
# ----------------------------



USE_GROQ = True
try:
    from groq import Groq
except Exception:
    USE_GROQ = False

try:
    from pypdf import PdfReader
except Exception as e:
    raise RuntimeError("pypdf yüklü değil. 'pip install pypdf' çalıştırın.") from e


# ============================================================
# 0) Genel yardımcılar
# ============================================================

def tr_normalize(text: str) -> str:
    if not text:
        return ""
    return (
        text.replace("İ", "i")
            .replace("I", "ı")
            .lower()
            .strip()
    )

def safe_print(obj):
    try:
        print(obj)
    except Exception:
        print(str(obj).encode("utf-8", errors="ignore").decode("utf-8"))

def now_ts():
    return time.strftime("%Y-%m-%d %H:%M:%S")

def clamp(x, a, b):
    return max(a, min(b, x))

# Basit tokenizasyon (TR/EN karışık idare eder)
_WORD_RE = re.compile(r"[a-zA-ZçğıöşüÇĞİÖŞÜ0-9]+", re.UNICODE)

def tokenize(text: str) -> List[str]:
    return _WORD_RE.findall(tr_normalize(text))

# ============================================================
# 1) PDF okuma + chunking
# ============================================================

def read_pdf_text(pdf_path: str, max_pages: Optional[int] = None) -> str:
    reader = PdfReader(pdf_path)
    texts = []
    n = len(reader.pages) if max_pages is None else min(len(reader.pages), max_pages)
    for i in range(n):
        page = reader.pages[i]
        t = page.extract_text() or ""
        texts.append(t)
    return "\n".join(texts)

def chunk_text(text: str, chunk_size: int = 900, overlap: int = 150) -> List[str]:
    """
    Basit karakter tabanlı chunk.
    PDF metinleri bazen satır satır kırık geldiği için bu yöntem stabil çalışır.
    """
    text = re.sub(r"\n{3,}", "\n\n", text)
    chunks = []
    start = 0
    L = len(text)
    while start < L:
        end = min(L, start + chunk_size)
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start = end - overlap
        if start < 0:
            start = 0
        if end == L:
            break
    return chunks

# ============================================================
# 2) Basit In-Memory Vector Store (TF-IDF benzeri)
#    - Dış bağımlılık olmadan Multi-RAG için yeterli
# ============================================================

class SimpleVectorStore:
    def __init__(self, name: str):
        self.name = name
        self.docs: List[Dict[str, Any]] = []
        self.df: Dict[str, int] = {}
        self.N = 0

    def add_documents(self, chunks: List[str], metadata: Dict[str, Any]):
        for idx, ch in enumerate(chunks):
            tokens = tokenize(ch)
            tf = {}
            for w in tokens:
                tf[w] = tf.get(w, 0) + 1
            uniq = set(tf.keys())
            for w in uniq:
                self.df[w] = self.df.get(w, 0) + 1
            self.docs.append({
                "text": ch,
                "tf": tf,
                "meta": {**metadata, "chunk_id": idx}
            })
            self.N += 1

    def _tfidf(self, tf: Dict[str, int]) -> Dict[str, float]:
        # log-tf * idf
        vec = {}
        for w, c in tf.items():
            idf = math.log((self.N + 1) / (self.df.get(w, 0) + 1)) + 1.0
            vec[w] = (1.0 + math.log(1 + c)) * idf
        return vec

    def _cosine(self, a: Dict[str, float], b: Dict[str, float]) -> float:
        inter = set(a.keys()) & set(b.keys())
        num = sum(a[w] * b[w] for w in inter)
        da = math.sqrt(sum(v*v for v in a.values()))
        db = math.sqrt(sum(v*v for v in b.values()))
        if da == 0 or db == 0:
            return 0.0
        return num / (da * db)

    def search(self, query: str, top_k: int = 4) -> List[Dict[str, Any]]:
        q_tokens = tokenize(query)
        q_tf = {}
        for w in q_tokens:
            q_tf[w] = q_tf.get(w, 0) + 1
        q_vec = self._tfidf(q_tf)

        scored = []
        for d in self.docs:
            d_vec = self._tfidf(d["tf"])
            s = self._cosine(q_vec, d_vec)
            scored.append((s, d))
        scored.sort(key=lambda x: x[0], reverse=True)
        results = []
        for s, d in scored[:top_k]:
            results.append({
                "score": float(s),
                "text": d["text"],
                "meta": d["meta"]
            })
        return results

# ============================================================
# 3) Multi-RAG Manager (SAFETY / REGULATION / SIGNS)
# ============================================================

class MultiRAG:
    def __init__(self):
        self.stores: Dict[str, SimpleVectorStore] = {}

    def ingest_pdf(self, pdf_path: str, doc_type: str, chunk_size: int = 900, overlap: int = 150, max_pages: Optional[int]=None):
        text = read_pdf_text(pdf_path, max_pages=max_pages)
        chunks = chunk_text(text, chunk_size=chunk_size, overlap=overlap)
        if doc_type not in self.stores:
            self.stores[doc_type] = SimpleVectorStore(doc_type)
        self.stores[doc_type].add_documents(
            chunks,
            metadata={"source": os.path.basename(pdf_path), "doc_type": doc_type}
        )

    def tool_search(self, query: str, doc_type: str, top_k: int = 4) -> Dict[str, Any]:
        """
        ReAct kuralı: Tool çıktı olarak "ham context" döndürür; final cevabı agent üretir.
        """
        if doc_type not in self.stores:
            return {"ok": False, "error": f"RAG store yok: {doc_type}", "results": []}
        results = self.stores[doc_type].search(query, top_k=top_k)

        # "yeterlilik" için basit eşik: en iyi skor çok düşükse yetersiz say
        best = results[0]["score"] if results else 0.0
        sufficient = best >= 0.08  # pratik eşik
        return {
            "ok": True,
            "doc_type": doc_type,
            "sufficient": sufficient,
            "best_score": best,
            "results": results
        }

# ============================================================
# 4) API Tools
# ============================================================

def live_traffic_api_tomtom(area_query: str) -> Dict[str, Any]:
    """
    TomTom Routing + Traffic örneği.
    - Gerçek kullanım için TOMTOM_API_KEY env değişkeni gerekir.
    - Bu fonksiyon "yoğunluk" mantığını temsili gösterir.
    """
    api_key = os.getenv("TOMTOM_API_KEY", "").strip()
    if not api_key:
        # Fallback mock
        mock = {
            "ok": True,
            "provider": "MOCK",
            "area": area_query,
            "summary": "API anahtarı bulunamadı; MOCK trafik durumu kullanıldı.",
            "traffic": {
                "congestion_pct": random.choice([0, 10, 25, 40, 60]),
                "status": random.choice(["Akıcı", "Yoğun", "Orta", "Çok Yoğun"])
            }
        }
        return mock

    # (İsteğe bağlı) Geocoding -> koordinat -> traffic flow vs.
    # Burada basit demo: TomTom Search API ile koordinat bul, sonra Flow API çağır.
    try:
        geo_url = "https://api.tomtom.com/search/2/geocode/{}.json".format(requests.utils.quote(area_query))
        geo_params = {"key": api_key, "limit": 1, "countrySet": "TR"}
        geo_r = requests.get(geo_url, params=geo_params, timeout=20)
        geo_r.raise_for_status()
        geo = geo_r.json()
        if not geo.get("results"):
            return {"ok": False, "provider": "TomTom", "error": "Konum bulunamadı."}
        pos = geo["results"][0]["position"]
        lat, lon = pos["lat"], pos["lon"]

        flow_url = f"https://api.tomtom.com/traffic/services/4/flowSegmentData/absolute/10/json"
        flow_params = {"key": api_key, "point": f"{lat},{lon}"}
        flow_r = requests.get(flow_url, params=flow_params, timeout=20)
        flow_r.raise_for_status()
        flow = flow_r.json()

        fsd = flow.get("flowSegmentData", {})
        # congestion: currentSpeed vs freeFlowSpeed gibi alanlardan türetebiliriz
        cur = fsd.get("currentSpeed", None)
        free = fsd.get("freeFlowSpeed", None)
        if cur is None or free is None or free == 0:
            congestion_pct = None
        else:
            congestion_pct = int(round((1 - (cur / free)) * 100))
            congestion_pct = clamp(congestion_pct, 0, 100)

        status = "Akıcı"
        if congestion_pct is not None:
            if congestion_pct >= 60:
                status = "Çok Yoğun"
            elif congestion_pct >= 35:
                status = "Yoğun"
            elif congestion_pct >= 15:
                status = "Orta"

        return {
            "ok": True,
            "provider": "TomTom",
            "area": area_query,
            "traffic": {
                "currentSpeed": cur,
                "freeFlowSpeed": free,
                "congestion_pct": congestion_pct,
                "status": status
            }
        }
    except Exception as e:
        return {"ok": False, "provider": "TomTom", "error": str(e)}

def math_tool(expression: str) -> Dict[str, Any]:
    """
    Güvenli eval (sadece sayılar ve + - * / ( ) . %).
    """
    expr = expression.strip()
    if not re.fullmatch(r"[0-9\.\+\-\*\/\(\)\s%]+", expr):
        return {"ok": False, "error": "İfadede izin verilmeyen karakter var."}
    try:
        # yüzde için: "50%" -> 0.5
        expr = re.sub(r"(\d+(?:\.\d+)?)\s*%", r"(\1/100)", expr)
        val = eval(expr, {"__builtins__": {}}, {})
        return {"ok": True, "value": val}
    except Exception as e:
        return {"ok": False, "error": str(e)}

# ============================================================
# 5) ReAct Agent (LLM Orkestrasyonu)
# ============================================================

@dataclass
class TraceStep:
    turn: int
    thought: str
    action: str
    observation: str

class SmartTrafficReActAgent:
    def __init__(self, rag: MultiRAG):
        self.rag = rag
        self.history: List[Dict[str, str]] = []

        # Groq LLM client (opsiyonel)
        self.groq_client = None
        self.groq_model = os.getenv("GROQ_MODEL", "llama-3.3-70b-versatile")
        if USE_GROQ:
            api_key = os.getenv("GROQ_API_KEY", "").strip()
            if api_key:
                self.groq_client = Groq(api_key=api_key)

        self.tools = {
            "rag_search": self._tool_rag_search,
            "live_traffic_api": self._tool_live_traffic,
            "math": self._tool_math,
        }

    # ---------- Tool wrappers ----------
    def _tool_rag_search(self, args: Dict[str, Any]) -> Dict[str, Any]:
        query = args.get("query", "")
        doc_type = args.get("doc_type", "")
        top_k = int(args.get("top_k", 4))
        return self.rag.tool_search(query=query, doc_type=doc_type, top_k=top_k)

    def _tool_live_traffic(self, args: Dict[str, Any]) -> Dict[str, Any]:
        area = args.get("area", "")
        return live_traffic_api_tomtom(area)

    def _tool_math(self, args: Dict[str, Any]) -> Dict[str, Any]:
        expr = args.get("expression", "")
        return math_tool(expr)

    # ---------- Prompt ----------
    def _system_prompt(self) -> str:
        # Agent.pdf: Tool'lar net tanımlanmalı; Thought-Action-Observation döngüsü; max_turns; halüsinasyon önleme.
        # Ayrıca: Her zaman Türkçe cevap.
        return f"""
Sen "Akıllı Ulaşım ve Trafik Optimizasyonu" için ReAct ajanısın.
Çalışma biçimin: THOUGHT -> ACTION -> OBSERVATION döngüsüyle ilerle; en sonda FINAL ANSWER ver.

Kurallar:
1) Araçlar dışında bilgi uydurma. Bir aracı (tool) yoksa çağırmaya çalışma.
2) RAG aracı (rag_search) "cevap" üretmez; sadece ham context getirir. Bu context'i analiz edip cevabı SEN yazarsın.
3) Bilgi yetersizse aynı aracı farklı sorguyla tekrar dene veya farklı araca geç (ör. canlı trafik için live_traffic_api).
4) Yanıt dili: Türkçe.
5) İlerleme sınırı: max_turns kadar adımda bitir.
6) Observation içinde gelen JSON'u kullanıcıya özetleyerek kullan; tüm JSON'u olduğu gibi dökme.

Kullanabileceğin araçlar:
- rag_search: PDF bilgi tabanlarında arama yapar.
  args: {{ "query": str, "doc_type": "SAFETY"|"REGULATION"|"SIGNS", "top_k": int }}
- live_traffic_api: Bir bölge için canlı trafik yoğunluğu bilgisini çeker.
  args: {{ "area": str }}
- math: Matematiksel hesaplama yapar.
  args: {{ "expression": str }}

HESAPLAMA KURALI (ÇOK ÖNEMLİ):
- “Hızın yarısı kadar metre” kuralında birim dönüşümü YAPMA.
- km/saat cinsinden verilen hız değeri DOĞRUDAN kullanılır.
- Örnek: 80 km/saat → 80 / 2 = 40 metre.
- m/s dönüşümü, 3.6 ile bölme veya başka fiziksel dönüşümler KULLANILMAZ.
- Bu tür sorularda math aracına gönderilecek ifade MUTLAKA:
  hız / 2
şeklinde olmalıdır.

CANLI TRAFİK KURALI:
- Soru canlı trafik içeriyorsa (şu an/anlık/yoğunluk), FINAL yazmadan ÖNCE MUTLAKA:
  ACTION: live_traffic_api {{"area":"<bölge>"}}
- Bu sorularda rag_search çağırmak zorunlu değildir.
- live_traffic_api çağrısından sonra 1 adım içinde FINAL ANSWER üret.


Cevap formatı:
THOUGHT: ...
ACTION: <tool_name> <json_args>
OBSERVATION: ...
... (gerekirse tekrar)
ÇOK ÖNEMLİ FORMAT:
- Eğer araç çağıracaksan ÇIKTI MUTLAKA şu iki satırdan oluşsun:
  THOUGHT: ...
  ACTION: tool_name {{"key":"value"}}
- Eğer cevap üreteceksen MUTLAKA:
  FINAL ANSWER: ...
Başka etiket kullanma.
JSON içinde tek tırnak kullanma.
ÇOK KRİTİK KURAL:
- Eğer soru "şu an", "anlık", "şimdi", "mevcut durum", "trafik durumu" gibi ifadeler içeriyorsa
  FINAL ANSWER üretmeden ÖNCE MUTLAKA live_traffic_api çağır.
- Bu tür sorularda tool çağırmadan cevap vermek YASAKTIR.


FINAL ANSWER: ...
""".strip()

    # ---------- LLM call ----------
    def _llm(self, prompt: str) -> str:
        """
        Groq varsa kullanır, yoksa "heuristic" fallback (demo).
        """
        if self.groq_client is not None:
            resp = self.groq_client.chat.completions.create(
                model=self.groq_model,
                messages=[
                    {"role": "system", "content": self._system_prompt()},
                    {"role": "user", "content": prompt},
                ],
                temperature=0.2,
            )
            return resp.choices[0].message.content

        # Fallback: basit kural tabanlı karar verici (LLM yoksa bile demo çalışsın)
        # Not: Bu mod sadece sistemi test etmek içindir.
        q = tr_normalize(prompt)
        if any(k in q for k in ["yoğunluk",  "anlık", "akıc", "tomtom", "google", "harita"]):
            return 'THOUGHT: Trafik yoğunluğu için canlı API gerekli.\nACTION: live_traffic_api {"area":"%s"}' % prompt
        if any(k in q for k in ["yağmur", "sis", "takip mesaf", "güvenli sürüş", "fren", "balata", "ıslak", "ıslan", "aquaplan", "kızak"]):

            return 'THOUGHT: Güvenli sürüş bilgisi SAFETY dokümanında.\nACTION: rag_search {"query":"%s","doc_type":"SAFETY","top_k":4}' % prompt
        if any(k in q for k in ["madde", "yönetmelik", "kanun", "ceza", "trafik işaret"]):
            # Ceza maddeleri her PDF’de olmayabilir; önce yönetmelik/kanun dene
            return 'THOUGHT: Yönetmelik/Kanun içinde aramalıyım.\nACTION: rag_search {"query":"%s","doc_type":"REGULATION","top_k":4}' % prompt
        return 'THOUGHT: Genel arama.\nACTION: rag_search {"query":"%s","doc_type":"REGULATION","top_k":4}' % prompt

    # ---------- ReAct parse ----------


    def _parse_action(self, text: str):
        # ACTION / Action / tool:json varyasyonlarını yakala
        m = re.search(
            r"(ACTION|Action)\s*:\s*([a-zA-Z_]+)\s*:?\s*(\{.*\})",
            text,
            re.DOTALL
        )
        if not m:
            return None, None, "ACTION bulunamadı."

        tool = m.group(2).strip()
        js = m.group(3).strip()

        # 1) JSON dene
        try:
            return tool, json.loads(js), ""
        except Exception:
            pass

        # 2) Tek tırnak vs için literal_eval dene
        try:
            return tool, ast.literal_eval(js), ""
        except Exception as e:
            return None, None, f"Arg parse hatası: {e}"


    # ---------- Main run ----------
    def answer(self, question: str, max_turns: int = 5, return_trace: bool = True) -> Dict[str, Any]:
        trace: List[TraceStep] = []
        tools_used: List[str] = []
        contexts_used: List[Dict[str, Any]] = []
        last_action = None

        prompt = question
        q_norm = tr_normalize(question)

        needs_rag = any(k in q_norm for k in [
            "yağmur", "sis", "takip mesafe", "güvenli sürüş", "fren", "balata",
            "ıslak", "ıslan", "su birik", "aquaplan", "kızak",
            "yönetmelik", "kanun", "madde", "trafik işaret", "levha", "elkitab"
        ])


        needs_math = any(k in q_norm for k in ["km", "km/s", "hız", "yarısı", "metre", "%"])

        needs_live_traffic = any(k in q_norm for k in ["şu an", "anlık", "şimdi", "trafik durumu", "yoğunluk"])


        for turn in range(1, max_turns + 1):
            llm_out = self._llm(prompt)

            # Eğer model doğrudan FINAL ANSWER verdiyse (bazı modeller yapar)
            m_final = re.search(
      r"(FINAL\s*ANSWER|FINAL|Answer|Cevap)\s*:\s*(.*)",
      llm_out,
      re.IGNORECASE | re.DOTALL
  )

            if m_final:
                if needs_live_traffic and "live_traffic_api" not in tools_used:
                    prompt = f"SORU: {question}\n\nÖnce MUTLAKA live_traffic_api çağır. FINAL üretme."
                    continue
                if needs_rag and "rag_search" not in tools_used:
                    prompt = f"SORU: {question}\n\nÖnce MUTLAKA rag_search çağır (uygun doc_type seç). FINAL üretme."
                    continue
                if needs_math and "math" not in tools_used:
                    prompt = f"SORU: {question}\n\nBu soru hesap gerektiriyor. Önce math aracını çağır. FINAL üretme."
                    continue

                # burada finali kabul et


      # ✅ Normal durum: FINAL ANSWER kabul edilir
                final = m_final.group(2).strip()
                return {
                    "ok": True,
                    "answer": final,
                    "trace": [t.__dict__ for t in trace] if return_trace else [],
                    "tools_used": tools_used,
                    "contexts_used": contexts_used
                }


            # Thought çek (opsiyonel)
            thought = ""
            m_th = re.search(r"THOUGHT:\s*(.*)", llm_out)
            if m_th:
                thought = m_th.group(1).strip()

            tool, args, err = self._parse_action(llm_out)
            if err or tool not in self.tools:
                # Tool uydurma/parse başarısız: güvenli şekilde bitir
                obs = f"Tool çağrısı çözümlenemedi veya tool yok. Hata: {err}"
                trace.append(TraceStep(turn, thought or "(yok)", "NONE", obs))
                return {
                    "ok": False,
                    "answer": "İstenen işlemi tamamlayamadım (araç çağrısı çözümlenemedi). Sorguyu biraz sadeleştirip tekrar deneyin.",
                    "trace": [t.__dict__ for t in trace] if return_trace else [],
                    "tools_used": tools_used,
                    "contexts_used": contexts_used
                }

            tools_used.append(tool)
            result = self.tools[tool](args)
            current_action = (tool, json.dumps(args, ensure_ascii=False, sort_keys=True))
            if current_action == last_action:
                prompt += "\nTekrar aynı aracı çağırdın. Artık FINAL ANSWER üret."
            last_action = current_action


            # Observation metni (LLM'e geri besleme)
            obs_text = json.dumps(result, ensure_ascii=False)[:6000]  # çok uzamasın

            # context kaydı (özellikle rag_search sonuçları)
            if tool == "rag_search" and result.get("ok"):
                contexts_used.append({
                    "doc_type": result.get("doc_type"),
                    "best_score": result.get("best_score"),
                    "top_snippets": [
                        {
                            "score": r.get("score"),
                            "source": r.get("meta", {}).get("source"),
                            "chunk_id": r.get("meta", {}).get("chunk_id"),
                            "text_preview": (r.get("text","")[:260] + "...")
                        }
                        for r in result.get("results", [])[:3]
                    ]
                })

            trace.append(TraceStep(
                turn=turn,
                thought=thought or "(yok)",
                action=f"{tool} {json.dumps(args, ensure_ascii=False)}",
                observation=obs_text[:800] + ("..." if len(obs_text) > 800 else "")
            ))

            # RAG yetersizse: agenti başka tool'a yönlendirecek ipucu ver
            # (Bu ipucu Agent.pdf'deki tool-switching fikrine uygun)
            guidance = ""
            if tool == "rag_search" and result.get("ok") and (not result.get("sufficient")):
                guidance = (
                    "\nNot: RAG sonuçları yetersiz görünüyor. "
                    "Sorguyu sadeleştirerek tekrar dene veya konu canlı veriyse live_traffic_api kullan."
                )
            if tool == "live_traffic_api" and result.get("ok"):
                guidance += "\nCANLI TRAFİK VERİSİ GELDİ. Artık tekrar tool çağırma; hemen FINAL ANSWER üret."


            prompt = (
                f"SORU: {question}\n\n"
                f"ÖNCEKİ ADIM ÇIKTISI:\n{llm_out}\n\n"
                f"OBSERVATION(JSON): {obs_text}\n"
                f"{guidance}\n\n"
                "Şimdi bir sonraki THOUGHT/ACTION adımını ver ya da FINAL ANSWER üret."
                 f"{guidance}\n\n"
                  "Zorunlu Kural: Eğer canlı trafik verisi geldiyse bir sonraki çıktın FINAL ANSWER olmak zorunda.\n"
                  "Şimdi FINAL ANSWER üret."
            )

        # max_turns bitti
        return {
            "ok": False,
            "answer": "Maksimum adım sayısına ulaşıldı (max_turns). Sorguyu daraltıp tekrar deneyin.",
            "trace": [t.__dict__ for t in trace] if return_trace else [],
            "tools_used": tools_used,
            "contexts_used": contexts_used
        }


# ============================================================
# 6) Kurulum: PDF -> Multi-RAG ingest
# ============================================================

def build_system() -> SmartTrafficReActAgent:
    rag = MultiRAG()

    # --- PDF yolları (senin upload ettiklerin) ---

    REG_YON_PDF = "/content/REG_trafik_yonetmeligi.pdf"
    REG_2918_PDF = "/content/REG_2918.pdf"
    SIGNS_PDF = "/content/REG_TrafikIsaretleriElKitabi.pdf"
    SAFETY_PDF = "/content/SAFETY_Guvenli_Surus_Teknikleri.pdf"

    # Multi-RAG: diğer pdfler multirag kısmında kullanılacak.
    # Agent.pdf sadece "tasarım rehberi"; istersek onu da "DESIGN" store'a koyabiliriz
    # ama cevap üretirken genelde gerekmez.
    rag.ingest_pdf(SAFETY_PDF, doc_type="SAFETY", chunk_size=950, overlap=160)
    rag.ingest_pdf(REG_YON_PDF, doc_type="REGULATION", chunk_size=950, overlap=160)
    rag.ingest_pdf(REG_2918_PDF, doc_type="REGULATION", chunk_size=950, overlap=160)  # aynı store'a ekledik
    rag.ingest_pdf(SIGNS_PDF, doc_type="SIGNS", chunk_size=950, overlap=160)

    agent = SmartTrafficReActAgent(rag=rag)
    return agent


# ============================================================
# 7) Benchmark / Senaryo Testleri
#    - İstenen: "RAG tek başına cevaplayamasın -> API'ye bağlansın"
# ============================================================

# ============================================================
# 7) Benchmark / Senaryo Testleri (TEK LİSTE)
# ============================================================

BENCHMARK_CASES = [

    # =========================
    # ORİJİNAL BENCHMARKLAR
    # =========================

    # (A) RAG ile çözülebilir (SAFETY/REG)
    {
        "id": "A1",
        "question": "Yağmurda güvenli sürüş için takip mesafesi kaç saniye olmalı ve neden?",
        "expected_tools": ["rag_search"]
    },
    {
        "id": "A2",
        "question": "Karayolları Trafik Yönetmeliğine göre araçlar arasındaki takip mesafesi nasıl tanımlanır?",
        "expected_tools": ["rag_search"]
    },
    {
        "id": "A3",
        "question": "Trafik işaretleri elkitabına göre işaret levhaları hangi ana gruplara ayrılır?",
        "expected_tools": ["rag_search"]
    },

    # (B) Multi-hop + Math (RAG + math)
    {
        "id": "B1",
        "question": "90 km/s hızla giden bir araç için 'hızın yarısı kadar metre' kuralına göre takip mesafesi kaç metredir?",
        "expected_tools": ["rag_search", "math"]
    },

    # (C) RAG yetmez -> canlı trafik (API)
    {
        "id": "C1",
        "question": "Esenyurt bölgesindeki anlık trafik yoğunluğu nedir?",
        "expected_tools": ["live_traffic_api"]
    },
    {
        "id": "C2",
        "question": "Şu anda Taksim çevresinde trafik çok yoğunsa, sürüş güvenliği açısından hangi önlemleri almalıyım? (Önce yoğunluğu kontrol et.)",
        "expected_tools": ["live_traffic_api", "rag_search"]
    },

    # =========================
    # ROUTING / INCIDENTS (ŞU AN TOOL EKSİKSE ÇALIŞMAZ)
    # Not: Şu an sadece live_traffic_api var; route_distance_api / incidents_api yoksa FAIL olur.
    # =========================
    {
        "id": "R1",
        "question": "Şu anki trafik yoğunluğuna göre Taksim ile Kadıköy arası kaç dakika sürer?",
        "expected_tools": ["live_traffic_api"]
    },
    {
        "id": "R2",
        "question": "Adana Turgut Özal Bulvarı çevresinde şu an trafik durumu nasıl, yol çalışması var mı?",
        "expected_tools": [ "live_traffic_api"]
    },

    # =========================
    # EK BENCHMARK SORULARI
    # =========================

    # --- SAFETY / REGULATION / SIGNS (RAG) ---
    {
        "id": "A4",
        "question": "Tehlikeli madde taşıyan araç sürücüleri, yerleşim birimleri dışındaki karayollarında diğer araçları en az kaç metre mesafeden takip etmek zorundadırlar?",
        "expected_tools": ["rag_search"]
    },
    {
        "id": "A5",
        "question": "Aracın su dolu derin bir çukura girmesi nedeniyle fren balatalarının ıslanması halinde frenin tutunma özelliği azalır. Bu durumda sürücü ne yapmalıdır?",
        "expected_tools": ["rag_search"]
    },
    {
        "id": "A6",
        "question": "Trafik işaretleri el kitabına göre trafik işaret levhaları hangi ana gruplara ayrılır?",
        "expected_tools": ["rag_search"]
    },

    # --- RAG + Math (Multi-hop) ---
    {
        "id": "B2",
        "question": "Güvenli sürüş kurallarına göre takip mesafesi hızla ilişkilidir. 95 km/s hızla seyreden bir araç için 'hızın yarısı kadar metre' kuralına göre takip mesafesini hesapla ve kuralı kısaca açıkla.",
        "expected_tools": ["rag_search", "math"]
    },
    {
        "id": "B3",
        "question": "Takip mesafesinin en az '2 saniye' olması gerektiği kuralına göre, 72 km/s hızla giden bir araç için güvenli takip mesafesi kaç metredir? (İpucu: önce m/s çevir, sonra 2 saniye ile çarp.)",
        "expected_tools": ["rag_search", "math"]
    },

    # --- Live traffic (API) ---
    {
        "id": "C3",
        "question": "Adana Sarıçam ilçesinde anlık trafik durumu nasıl ve trafik akışı hangi seviyededir?",
        "expected_tools": ["live_traffic_api"]
    },

    # --- Live traffic + RAG (koşullu güvenlik) ---
    {
        "id": "C4",
        "question": "Önce Esenyurt bölgesindeki anlık trafik yoğunluğunu kontrol et. Eğer trafik yoğunsa, güvenli sürüş için hangi önlemler alınmalıdır?",
        "expected_tools": ["live_traffic_api", "rag_search"]
    },
    {
        "id": "C5",
        "question": "Adana Sarıçam’da trafik durumu yoğun ise, takip mesafesi ve hız açısından sürücüler nasıl davranmalıdır? (Önce anlık yoğunluğu kontrol et.)",
        "expected_tools": ["live_traffic_api", "rag_search"]
    },

    # --- RAG (strateji / kavramsal) ---
    {
        "id": "D1",
        "question": "Trafik yoğunluğu değişken olan bölgelerde sürücüler riskleri azaltmak için nasıl bir sürüş stratejisi izlemelidir?",
        "expected_tools": ["rag_search"]
    },
    {
        "id": "D2",
        "question": "Anlık trafik verisinin olmadığı durumlarda, akıllı ulaşım sistemleri sürüş güvenliğini nasıl optimize edebilir?",
        "expected_tools": ["rag_search"]
    },

    # =========================
    # ZOR (HARD) BENCHMARKLAR
    # =========================

    {
        "id": "H1",
        "question": "Önce Esenyurt için anlık trafik yoğunluğunu kontrol et. Eğer yoğunluk %35 ve üzeriyse, güvenli sürüş için 3 maddelik önlem listesi çıkar ve 80 km/s hız için 'hızın yarısı' kuralına göre takip mesafesini hesapla.",
        "expected_tools": ["live_traffic_api", "rag_search", "math"]
    },
    {
        "id": "H2",
        "question": "Eğer canlı trafik verisi alınamazsa (API hatası/anahtar yok), sürüş güvenliğini artırmak için sistem hangi stratejiyle karar vermelidir? 4 adımda açıkla.",
        "expected_tools": ["rag_search"]
    },
    {
        "id": "H3",
        "question": "2 saniye kuralını kısaca açıkla ve 108 km/s hızla giden bir araç için güvenli takip mesafesini metre cinsinden hesapla.",
        "expected_tools": ["rag_search", "math"]
    },
    {
        "id": "H4",
        "question": "Trafik işaret levhalarının ana gruplarını belirt ve sürücünün bu grupları bilmesinin güvenlik açısından 2 faydasını açıkla.",
        "expected_tools": ["rag_search"]
    },
]

def run_benchmark(agent: SmartTrafficReActAgent, max_turns: int = 5):
    results = []
    for case in BENCHMARK_CASES:
        out = agent.answer(case["question"], max_turns=max_turns, return_trace=True)
        results.append({
            "id": case["id"],
            "ok": out["ok"],
            "question": case["question"],
            "answer_preview": (out["answer"][:220] + "...") if out.get("answer") else "",
            "tools_used": out.get("tools_used", []),
            "expected_tools": case["expected_tools"],
            "trace": out.get("trace", []),
            "contexts_used": out.get("contexts_used", [])
        })
    return results

def benchmark_summary(results):
    total = len(results)
    ok_count = sum(1 for r in results if r.get("ok"))
    tool_match_count = sum(
        1 for r in results
        if set(r.get("expected_tools", [])).issubset(set(r.get("tools_used", [])))
    )

    avg_turns = round(sum(len(r.get("trace", [])) for r in results) / total, 2) if total else 0

    return {
        "Toplam Senaryo": total,
        "Cevap Başarı (ok) %": round(100 * ok_count / total, 2) if total else 0,
        "Tool Doğruluk (expected ⊆ used) %": round(100 * tool_match_count / total, 2) if total else 0,
        "Ortalama Adım Sayısı": avg_turns
    }


# ============================================================
# 8) Çalıştır
# ============================================================

if __name__ == "__main__":
    agent = build_system()

    # Tek soru dene
    demo_q = "Yağmurda takip mesafesi ile ilgili kural nedir?"
    demo = agent.answer(demo_q, max_turns=5, return_trace=True)

    print("\n================= DEMO =================")
    print("Soru:", demo_q)
    print("Cevap:", demo["answer"])
    print("Tools:", demo["tools_used"])
    print("Contexts Used (özet):", json.dumps(demo["contexts_used"], ensure_ascii=False, indent=2))
    print("Trace (Thought/Action/Observation):")
    for step in demo["trace"]:
        print(f"\n--- Turn {step['turn']} ---")
        print("THOUGHT:", step["thought"])
        print("ACTION:", step["action"])
        print("OBSERVATION:", step["observation"])


    # Benchmark çalıştır
    bench = run_benchmark(agent, max_turns=5)

    print("\n================= BENCHMARK SUMMARY =================")
    summary = benchmark_summary(bench)
    for k, v in summary.items():
        print(f"{k}: {v}")

    print("\n================= BENCHMARK =================")
    for r in bench:
        print(f"\n--- Case {r['id']} ---")
        print("OK:", r["ok"])
        print("Tools Used:", r["tools_used"])
        print("Answer Preview:", r["answer_preview"])
        #İstersen trace'i de detaylı bas:
        print(json.dumps(r["trace"], ensure_ascii=False, indent=2))



================= DEMO =================
Soru: Yağmurda takip mesafesi ile ilgili kural nedir?
Cevap: Yağmurda takip mesafesi 3-4 saniyeye çıkarılmalıdır.
Tools: ['rag_search']
Contexts Used (özet): [
  {
    "doc_type": "SAFETY",
    "best_score": 0.1963672236139452,
    "top_snippets": [
      {
        "score": 0.1963672236139452,
        "source": "SAFETY_Guvenli_Surus_Teknikleri.pdf",
        "chunk_id": 3,
        "text_preview": "k\ntemizlenmelidir.\nÇEVRE İLE İLGİLİ GÜVENLİ SÜRÜŞ TEKNİKLERİ\n• Yol yüzeyinde biriken toz ve yağlar ilk yağan yağmurla\nbirlikte yolu daha kayganlaştırır.\n• Bol yağan yağmurda yol yüzeyi temizlenir, ancak yol\nüzerinde biriken su aquaplaning’e (suda kızaklamaya)\n..."
      }
    ]
  }
]
Trace (Thought/Action/Observation):

--- Turn 2 ---
THOUGHT: Yağmurda takip mesafesi ile ilgili kuralı öğrenmek için rag_search aracını kullanacağım. Uygun doc_type olarak "SAFETY" seçeceğim.
ACTION: rag_search {"query": "yağmurda takip mesafesi", "doc_type": "SAFET

In [5]:
!pip -q install groq pandas

from groq import Groq
import os

client = Groq(api_key=os.getenv("GROQ_API_KEY","").strip())
import time, json
import pandas as pd

def judge_with_llm(question, agent_answer, expected_tools, used_tools, contexts_used):
    """
    LLM-as-a-Judge: Senin proje bağlamına göre kalite puanı (0-100).
    """
    ctx_hint = ""
    if contexts_used:
        # sadece kaynak isimlerini özet geç
        sources = []
        for c in contexts_used:
            for sn in c.get("top_snippets", []):
                if sn.get("source"):
                    sources.append(sn["source"])
        sources = list(dict.fromkeys(sources))[:5]
        if sources:
            ctx_hint = f"\nContexts (sources): {', '.join(sources)}"

    judge_prompt = f"""
You are an impartial Quality Assurance Auditor for a Smart Transportation ReAct Agent.

Evaluation Criteria:
1) Tool Correctness: Did the agent use the right tools for this question?
   - Expected tools: {expected_tools}
   - Used tools: {used_tools}
2) Answer Quality: Does the final answer address the question clearly and correctly?
3) Source Fidelity: If RAG was used, is the answer consistent with retrieved context (no hallucination)?

User Question: "{question}"
Agent Final Answer: "{agent_answer}"
{ctx_hint}

TASK: Return a single integer score between 0 and 100.
Return ONLY the integer number, no text.
""".strip()

    try:
        resp = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role":"user","content":judge_prompt}],
            temperature=0
        )
        score_text = resp.choices[0].message.content.strip()
        digits = ''.join(ch for ch in score_text if ch.isdigit())
        return int(digits) if digits else 0
    except Exception as e:
        print("Judge error:", e)
        return 0


def run_llm_judge_benchmark(agent, cases, max_turns=5, save_csv_path="/content/final_performance_report.csv"):
    final_results = []
    print(f"✅ STARTING LLM-JUDGE BENCHMARK ({len(cases)} cases)\n" + "="*70)

    for case in cases:
        q = case["question"]
        exp_tools = case.get("expected_tools", [])

        print(f"🔍 {case['id']}: {q}")
        start = time.time()

        out = agent.answer(q, max_turns=max_turns, return_trace=True)
        latency = round(time.time() - start, 2)

        ans = out.get("answer","")
        used_tools = out.get("tools_used", [])
        contexts_used = out.get("contexts_used", [])

        # tool doğruluk
        tool_ok = set(exp_tools).issubset(set(used_tools))

        # judge score
        score = judge_with_llm(q, ans, exp_tools, used_tools, contexts_used)

        final_results.append({
            "ID": case["id"],
            "Question": q,
            "OK": out.get("ok", False),
            "Expected_Tools": ", ".join(exp_tools),
            "Used_Tools": ", ".join(used_tools),
            "Tool_Correct": tool_ok,
            "Judge_Score": score,
            "Latency_Sec": latency,
            "Steps": len(out.get("trace", [])),
        })

        print(f"   ✅ Tool_Correct={tool_ok} | Judge={score} | Latency={latency}s | Steps={len(out.get('trace', []))}")

    df = pd.DataFrame(final_results)

    # summary
    avg_score = round(df["Judge_Score"].mean(), 2) if len(df) else 0
    avg_latency = round(df["Latency_Sec"].mean(), 2) if len(df) else 0
    tool_acc = round(100 * df["Tool_Correct"].mean(), 2) if len(df) else 0
    ok_rate = round(100 * df["OK"].mean(), 2) if len(df) else 0

    summary_row = {
        "ID": "SUMMARY",
        "Question": "AVERAGES",
        "OK": f"{ok_rate}%",
        "Expected_Tools": "-",
        "Used_Tools": "-",
        "Tool_Correct": f"{tool_acc}%",
        "Judge_Score": avg_score,
        "Latency_Sec": avg_latency,
        "Steps": round(df["Steps"].mean(), 2) if len(df) else 0
    }
    df_out = pd.concat([df, pd.DataFrame([summary_row])], ignore_index=True)

    df_out.to_csv(save_csv_path, index=False)

    print("\n" + "="*70)
    print("📌 LLM-JUDGE BENCHMARK SUMMARY")
    print(f"OK Rate: {ok_rate}%")
    print(f"Tool Accuracy: {tool_acc}%")
    print(f"Avg Judge Score: {avg_score}")
    print(f"Avg Latency: {avg_latency}s")
    print(f"Saved CSV: {save_csv_path}")
    print("="*70)

    display(df_out[["ID","OK","Tool_Correct","Judge_Score","Latency_Sec","Steps"]])
    return df_out


# ✅ Çalıştır (agent ana koddan geliyor olmalı)
agent = build_system()
df_report = run_llm_judge_benchmark(agent, BENCHMARK_CASES, max_turns=5, save_csv_path="/content/final_performance_report.csv")


✅ STARTING LLM-JUDGE BENCHMARK (22 cases)
🔍 A1: Yağmurda güvenli sürüş için takip mesafesi kaç saniye olmalı ve neden?
   ✅ Tool_Correct=True | Judge=100 | Latency=1.81s | Steps=1
🔍 A2: Karayolları Trafik Yönetmeliğine göre araçlar arasındaki takip mesafesi nasıl tanımlanır?
   ✅ Tool_Correct=True | Judge=100 | Latency=1.72s | Steps=1
🔍 A3: Trafik işaretleri elkitabına göre işaret levhaları hangi ana gruplara ayrılır?
   ✅ Tool_Correct=True | Judge=100 | Latency=2.21s | Steps=1
🔍 B1: 90 km/s hızla giden bir araç için 'hızın yarısı kadar metre' kuralına göre takip mesafesi kaç metredir?
   ✅ Tool_Correct=True | Judge=100 | Latency=2.0s | Steps=2
🔍 C1: Esenyurt bölgesindeki anlık trafik yoğunluğu nedir?
   ✅ Tool_Correct=True | Judge=100 | Latency=1.27s | Steps=1
🔍 C2: Şu anda Taksim çevresinde trafik çok yoğunsa, sürüş güvenliği açısından hangi önlemleri almalıyım? (Önce yoğunluğu kontrol et.)
   ✅ Tool_Correct=False | Judge=80 | Latency=2.05s | Steps=1
🔍 R1: Şu anki trafik yoğunluğuna 

,ID,OK,Tool_Correct,Judge_Score,Latency_Sec,Steps
0,A1,True,True,100.00,1.81,1.00
1,A2,True,True,100.00,1.72,1.00
2,A3,True,True,100.00,2.21,1.00
3,B1,True,True,100.00,2.00,2.00
4,C1,True,True,100.00,1.27,1.00
5,C2,True,False,80.00,2.05,1.00
6,R1,True,True,80.00,1.93,1.00
7,R2,True,True,100.00,1.40,1.00
8,A4,False,True,0.00,2.87,3.00
9,A5,True,True,100.00,1.48,1.00


In [7]:
    demo_q = "Tehlikeli madde taşıyan araç sürücüleri, yerleşim birimleri dışındaki karayollarında diğer araçları, en az  kaç metre mesafeden takip etmek zorundadırlar? "
    demo = agent.answer(demo_q, max_turns=5, return_trace=True)
    print("\n================= DEMO =================")
    print("Soru:", demo_q)
    print("Cevap:", demo["answer"])
    print("Tools:", demo["tools_used"])
    print("Contexts Used (özet):", json.dumps(demo["contexts_used"], ensure_ascii=False, indent=2))
    print("Trace (Thought/Action/Observation):")
    for step in demo["trace"]:
        print(f"\n--- Turn {step['turn']} ---")
        print("THOUGHT:", step["thought"])
        print("ACTION:", step["action"])
        print("OBSERVATION:", step["observation"])


================= DEMO =================
Soru: Tehlikeli madde taşıyan araç sürücüleri, yerleşim birimleri dışındaki karayollarında diğer araçları, en az  kaç metre mesafeden takip etmek zorundadırlar? 
Cevap: Tehlikeli madde taşıyan araç sürücüleri, yerleşim birimleri dışındaki karayollarında diğer araçları en az 50 metre mesafeden takip etmek zorundadırlar.
Tools: ['rag_search', 'math']
Contexts Used (özet): [
  {
    "doc_type": "REGULATION",
    "best_score": 0.1449981483382254,
    "top_snippets": [
      {
        "score": 0.1449981483382254,
        "source": "REG_trafik_yonetmeligi.pdf",
        "chunk_id": 295,
        "text_preview": "lecekleri kadar açıklıkla r bulundurmak \nzorundadırlar. Bu açıklıklar, kol veya gruba dahil araçların azami hızlarına göre takip mesafesinden az \nolmayacaktır. \nTehlikeli madde taşıyan araç sürücüleri, yerleşim birimleri dışındaki karayollarında diğer \naraçlar..."
      }
    ]
  }
]
Trace (Thought/Action/Observation):

--- Turn 2 ---
THOUG

In [7]:
    demo_q = "Aracın su dolu derin bir çukura girmesi nedeniyle,fren balatalarının ıslanması halinde frenin tutunma özelliğinin azaldığı durumlarda hangisi yapılır? "
    demo = agent.answer(demo_q, max_turns=5, return_trace=True)
    print("\n================= DEMO =================")
    print("Soru:", demo_q)
    print("Cevap:", demo["answer"])
    print("Tools:", demo["tools_used"])
    print("Contexts Used (özet):", json.dumps(demo["contexts_used"], ensure_ascii=False, indent=2))
    print("Trace (Thought/Action/Observation):")
    for step in demo["trace"]:
        print(f"\n--- Turn {step['turn']} ---")
        print("THOUGHT:", step["thought"])
        print("ACTION:", step["action"])
        print("OBSERVATION:", step["observation"])


================= DEMO =================
Soru: Aracın su dolu derin bir çukura girmesi nedeniyle,fren balatalarının ıslanması halinde frenin tutunma özelliğinin azaldığı durumlarda hangisi yapılır? 
Cevap: Aracın su dolu derin bir çukura girmesi nedeniyle fren balatalarının ıslanması halinde frenin tutunma özelliğinin azaldığı durumlarda, fren balatalarının kuruması için birkaç sefer kısa sürelerde hafifçe frene basılıp bırakılması yapılır.
Tools: ['rag_search']
Contexts Used (özet): [
  {
    "doc_type": "SAFETY",
    "best_score": 0.22905347437214707,
    "top_snippets": [
      {
        "score": 0.22905347437214707,
        "source": "SAFETY_Guvenli_Surus_Teknikleri.pdf",
        "chunk_id": 4,
        "text_preview": "e çıkarılmalıdır.\nÇEVRE İLE İLGİLİ GÜVENLİ SÜRÜŞ TEKNİKLERİ\n• Yağmurda parke yollar, boyalı yol Şeritleri ve biriken\nçamurlardan ötürü yol kenarları kayganlaşır.\n• Su dolu derin çukurlar fren balatalarının ıslanmasına ve\nfrenin iyi tutmamasına neden olur. Deri.

In [12]:
    demo_q = " Adana Sarıçam ilçesinde anlık trafik durumu nasıl ve trafik akışı hangi seviyededir?  "
    demo = agent.answer(demo_q, max_turns=5, return_trace=True)
    print("\n================= DEMO =================")
    print("Soru:", demo_q)
    print("Cevap:", demo["answer"])
    print("Tools:", demo["tools_used"])
    print("Contexts Used (özet):", json.dumps(demo["contexts_used"], ensure_ascii=False, indent=2))
    print("Trace (Thought/Action/Observation):")
    for step in demo["trace"]:
        print(f"\n--- Turn {step['turn']} ---")
        print("THOUGHT:", step["thought"])
        print("ACTION:", step["action"])
        print("OBSERVATION:", step["observation"])


================= DEMO =================
Soru:  Adana Sarıçam ilçesinde anlık trafik durumu nasıl ve trafik akışı hangi seviyededir?  
Cevap: Adana Sarıçam ilçesinde anlık trafik durumu "Orta" seviyededir ve trafik akışı %17 oranında yoğundur. Mevcut hız 55 km/saat, serbest akış hızı ise 66 km/saat olarak raporlanmıştır.
Tools: ['live_traffic_api']
Contexts Used (özet): []
Trace (Thought/Action/Observation):

--- Turn 2 ---
THOUGHT: Adana Sarıçam ilçesinde anlık trafik durumunu öğrenmek için canlı trafik API'sini kullanacağım.
ACTION: live_traffic_api {"area": "Adana Sarıçam"}
OBSERVATION: {"ok": true, "provider": "TomTom", "area": "Adana Sarıçam", "traffic": {"currentSpeed": 55, "freeFlowSpeed": 66, "congestion_pct": 17, "status": "Orta"}}
